In [1]:
import torch
import numpy as np
import pickle

from HMM_Likelyhood import HMM_log_likelihood,Optimizationloop, InitializeParameters,Accumulated_HHM_LL
import LogLikelihoodTorch


with open('.DataPhase.pickle','rb') as f:
    X = pickle.load(f) 
X_tensor = X_tensor = torch.from_numpy(X).to(torch.float64)
#X_tensor = torch.reshape(X_tensor.T,(-1,90,330))
torch.set_default_dtype(torch.float64)

In [15]:
def HMM_Optimization(X,kappa,mu,Tk,Pinit,learning_rate,Subjects,n_iters,K):
    

    # Initialize MixtureModel of MWD's to gain e good estimate for HMM
    pi =torch.tensor([1/K for _ in range(K)],requires_grad=True)

    Parameters_torch = [
    {'params':pi},
    {'params':kappa},
    {'params':mu},
    ]
    AdamTorch = torch.optim.Adam(Parameters_torch,lr=0.8)
    Pi_fit,Kappa_fit,mu_fit = LogLikelihoodTorch.Optimizationloop(X,[pi,kappa,mu],lose=LogLikelihoodTorch.log_likelihood,Optimizer=AdamTorch,n_iters=2500,K=K)


    Parameters_HMM= [
    {'params':Pinit},
    {'params':Kappa_fit},
    {'params':mu_fit},
    {'params':Tk},
    ]

    AdamHMM=torch.optim.Adam(Parameters_HMM,lr=learning_rate)
    AdamHMM.zero_grad()
    Pinit_fit,Kappa_fit,mu_fit,Tk_fit = Optimizationloop(X_tensor,[Pinit,Kappa_fit,mu_fit,Tk],lose=Accumulated_HHM_LL,Optimizer=AdamHMM,n=Subjects,n_iters=n_iters,K=K)



    return Pinit_fit,Kappa_fit,mu_fit,Tk_fit

In [16]:
K=5
p=90
Subjects = 199

kappa,mu,Tk,Pinit = InitializeParameters(Subjects,p,K)
Pinit_fit,Kappa_fit,mu_fit,Tk_fit = HMM_Optimization(X_tensor,kappa,mu,Tk,Pinit,0.4,Subjects,500,K)

epoch 1; Log-Likelihood = -5529651.945966571
epoch 101; Log-Likelihood = -6211214.377157628
epoch 201; Log-Likelihood = -6211170.850741458
epoch 301; Log-Likelihood = -6282766.537617215
epoch 401; Log-Likelihood = -6323837.7539714435
epoch 501; Log-Likelihood = -6403701.06292844
epoch 601; Log-Likelihood = -6453248.411966923
epoch 701; Log-Likelihood = -6502291.700331236
epoch 801; Log-Likelihood = -6549304.375067535
epoch 901; Log-Likelihood = -6594572.036931935
epoch 1001; Log-Likelihood = -6638415.278768824
epoch 1101; Log-Likelihood = -6681177.877256498
epoch 1201; Log-Likelihood = -6706960.088115221
epoch 1301; Log-Likelihood = -6706963.868642068
epoch 1401; Log-Likelihood = -6706968.315672187
epoch 1501; Log-Likelihood = -6706968.208540375
epoch 1601; Log-Likelihood = -6706963.045632873
epoch 1701; Log-Likelihood = -6706928.219576255
epoch 1801; Log-Likelihood = -6706965.583933014
epoch 1901; Log-Likelihood = -6706964.818580173
epoch 2001; Log-Likelihood = -6706962.442578779
epoc

AttributeError: 'Tensor' object has no attribute 'zero_grad'

In [2]:
# Initialize parameters to tune:
K=5
p=90
Subjects = 199

kappa,mu,Tk,Pinit = InitializeParameters(Subjects,p,K)

learning_rate = 0.4
n_iters = 500

torch.autograd.set_detect_anomaly(False)
Parameters = [
    {'params':Pinit},
    {'params':kappa},
    {'params':mu},
    {'params':Tk},
]

Adam = torch.optim.Adam(Parameters,lr=learning_rate)


# pi.to(device)
# mu.to(device)
# kappa.to(device)
# Tk.to(device)
# X_tensor.to(device)

In [3]:

Pinit_fit,Kappa_fit,mu_fit,Tk_fit = Optimizationloop(X_tensor,[Pinit,kappa,mu,Tk],lose=Accumulated_HHM_LL,Optimizer=Adam,n=Subjects,n_iters=n_iters,K=K)

epoch 1; Log-Likelihood = -4801213.798400863
epoch 2; Log-Likelihood = -4807727.016851751
epoch 3; Log-Likelihood = -4827094.726926308
epoch 4; Log-Likelihood = -4857556.9722787645
epoch 5; Log-Likelihood = -4892370.691269308
epoch 6; Log-Likelihood = -4928949.282963748
epoch 7; Log-Likelihood = -4966246.614706213
epoch 8; Log-Likelihood = -5003464.027385467
epoch 9; Log-Likelihood = -5040461.327512468
epoch 10; Log-Likelihood = -5078122.055316435
epoch 11; Log-Likelihood = -5116249.622801734
epoch 12; Log-Likelihood = -5154596.4666617755
epoch 13; Log-Likelihood = -5193896.158077509
epoch 14; Log-Likelihood = -5233899.786782889
epoch 15; Log-Likelihood = -5273806.036404819
epoch 16; Log-Likelihood = -5313826.376449587
epoch 17; Log-Likelihood = -5353806.501921997
epoch 18; Log-Likelihood = -5394141.354606332
epoch 19; Log-Likelihood = -5434646.57195493
epoch 20; Log-Likelihood = -5475328.155632332
epoch 21; Log-Likelihood = -5516655.237475568
epoch 22; Log-Likelihood = -5557862.134612

In [6]:

Softmax = torch.nn.Softmax(0)
Softplus = torch.nn.Softplus()

kappa_est = Softplus(Kappa_fit)
#kappa_est = Kappa_fit
mu_est = mu /torch.sqrt((mu_fit * mu_fit).sum(axis=0))

In [8]:
print(Softmax(Tk_fit[0]))

tensor([[1.1648e-12, 1.0998e-37, 5.5391e-09, 3.9402e-39, 3.4205e-14],
        [1.0000e+00, 5.1295e-70, 1.0000e+00, 1.0000e+00, 1.0000e+00],
        [2.4183e-12, 3.0146e-37, 1.0931e-09, 7.6072e-39, 3.1097e-14],
        [1.4001e-21, 1.0000e+00, 1.3445e-13, 6.3468e-70, 2.7136e-21],
        [1.3142e-12, 1.3332e-37, 3.3300e-10, 4.4944e-40, 1.2628e-14]],
       grad_fn=<SoftmaxBackward0>)


In [10]:
import pandas as pd
labels = pd.read_csv("AALlabels.csv")
label=labels['labels'].values
import plotly.express as px
for model in range(K):
    print(f"Model {model}:  , kappa = {kappa_est[model]}")
    fig = px.bar(y=mu_est[:,model].detach().numpy(),x=label)
    #fig.update_layout(width=300)
    fig.show()

Model 0:  , kappa = 52.878553669276066


Model 1:  , kappa = 51.491669361172775


Model 2:  , kappa = 35.151607122095555


Model 3:  , kappa = 47.531134485465046


Model 4:  , kappa = 53.37865110437062
